In [93]:
import os
import pickle
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import random
import shutil
# from plot import * #load_array

### Explanation
- For each region get the label of each timestamp
- The mask in the region is (0,1) 0 being invalid areas, such as farmlands, 1 being valid areas such as forest area
- Within the forest area, there are deforested regions
- These regions are taken into consideration by creating a third mask, 2. The mask is generated using the mask channel (11th channel) in the region data, and the 1st and 2nd channels (ID, time stamp) in the areas without trees data
- If a pixel is 1 in the region mask (meaning it is part of the valid region) and if it is greater than 0 in the ID channel of the without trees data (meaning it is a deforested area/clearing) and if it is a specific date based on the file name, the clearing was created at that time stamp. 
- I think I should do less than equal to to take into consideration the deforested areas that already occured in previous time steps and not just new ones in that time step

- Load region data (x, y, 11) and without tree data (x, y, 4) files 
- Concatenate the 'id','timestamp' and 'veg cover' channels to all 'region_..._final_data' arrays
- 'region_..._final_data' arrays now have a shape of (x, y, 13)

Do not apppend areas without trees to dictionary so len should be 6 and not 8

In [85]:
folder_path = "/home/k45848/multispectral-imagery-segmentation/data"
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith(".pkl")]
file_paths

['/home/k45848/multispectral-imagery-segmentation/data/region_2_east_2023_05_31_final_data.pkl',
 '/home/k45848/multispectral-imagery-segmentation/data/region_2_east_2024_02_25_final_data.pkl',
 '/home/k45848/multispectral-imagery-segmentation/data/areas_without_trees_east_raster_final.pkl',
 '/home/k45848/multispectral-imagery-segmentation/data/region_1_west_2023_09_26_final_data.pkl',
 '/home/k45848/multispectral-imagery-segmentation/data/region_1_west_2024_02_28_final_data.pkl',
 '/home/k45848/multispectral-imagery-segmentation/data/areas_without_trees_west_raster_final.pkl',
 '/home/k45848/multispectral-imagery-segmentation/data/region_2_east_2023_09_28_final_data.pkl',
 '/home/k45848/multispectral-imagery-segmentation/data/region_1_west_2023_05_31_final_data.pkl']

In [86]:
def load_array(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)
    
folder_path = "/home/k45848/multispectral-imagery-segmentation/data"
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith(".pkl")]

arrays = {}
for path in file_paths:
    array = load_array(path)

    # Extract file name without extension as key
    key = os.path.splitext(os.path.basename(path))[0]
    
    # Store array in dictionary
    arrays[key] = array

# Iterate through the dictionary keys
for key in arrays.keys():
    # Check if the key contains 'without_trees'
    if 'without_trees' in key:
        # Extract cardinal point from the key
        cardinal_point = key.split('_')[-3]
        
        # Find matching keys without 'without_trees' and the same cardinal point
        matching_keys = [k for k in arrays.keys() if cardinal_point in k and 'without_trees' not in k]
        
        # Iterate through matching keys and update arrays
        for match_key in matching_keys:
            # Add the first two channels from the 'without_trees' array to the matching array
            arrays[match_key] = np.concatenate([arrays[match_key], arrays[key][...]], axis=-1)

mask = 'without_trees'
arrays = {key: value for key, value in arrays.items() if mask not in key}

for key, value in arrays.items():
    print(key, value.shape)            

region_2_east_2023_05_31_final_data (746, 744, 15)
region_2_east_2024_02_25_final_data (746, 744, 15)
region_1_west_2023_09_26_final_data (931, 932, 15)
region_1_west_2024_02_28_final_data (931, 932, 15)
region_2_east_2023_09_28_final_data (746, 744, 15)
region_1_west_2023_05_31_final_data (931, 932, 15)


- Contradictry to the previous explanation, I concatenated all channels in the 'areas without trees' files to their corresponding regional image array
- Shape = (x,y,15)
- array[:,:10] = 10 spectral channels of image
- array[10] = mask created by Denise 1 (valid: forest area), and 0 (invalid: urban areas)
- array[11] = id (areas without trees)
- array[12] = timestamp (areas without trees)
- array[13] = type (areas without trees)
- array[14] = veg_cover (areas without trees)

- Apply condition to get the regions that are deforested for each time stamp
- So now channel 11 has 3 unique values (0,1,2) rather than 2 (0,1)

In [87]:
for key, value in arrays.items():
    print(np.unique(value[...,14], return_counts=True))
 

(array([0., 1., 2., 3.], dtype=float32), array([547678,   6745,    399,    202]))
(array([0., 1., 2., 3.], dtype=float32), array([547678,   6745,    399,    202]))
(array([0., 1., 2., 3., 4.], dtype=float32), array([860466,   4442,    908,    445,   1431]))
(array([0., 1., 2., 3., 4.], dtype=float32), array([860466,   4442,    908,    445,   1431]))
(array([0., 1., 2., 3.], dtype=float32), array([547678,   6745,    399,    202]))
(array([0., 1., 2., 3., 4.], dtype=float32), array([860466,   4442,    908,    445,   1431]))


In [88]:
items = list(arrays.items())#[:-1]

# Add deforested class 2 to channel 11
for key, array in items:
    time_stamp_match = re.search(r'(\d{4})_(\d{2})_(\d{2})', key)

    if time_stamp_match:
        ts = float(time_stamp_match.group(1)[2:] + time_stamp_match.group(2) + time_stamp_match.group(3))

    condition = (array[..., 10] == 1) & (array[..., 11] > 0) & (array[..., 12] <= ts)
    
    array[condition, 10] = 2


    # print(np.unique(array[..., -1], return_counts=True))

    # Make a new channel where deforested class pixels are split into their different vegetation cover
    # Add a new channel, initialise to zero
    veg_mask = np.zeros(array.shape[:-1] +(1,))
    array = np.concatenate((array, veg_mask), axis=-1)

    condition_1 = (array[..., 10] == 1)
    array[condition_1, 15] = 5

    condition_2 = (array[..., 10] == 2) & (array[..., 14] == 1)
    array[condition_2, 15] = 1

    condition_3 = (array[..., 10] == 2) & (array[..., 14] == 2)
    array[condition_3, 15] = 2

    condition_4 = (array[..., 10] == 2) & (array[..., 14] == 3)
    array[condition_4, 15] = 3

    condition_5 = (array[..., 10] == 2) & (array[..., 14] == 4)
    array[condition_5, 15] = 4

    """At this point array has 16 channels: 10 spectral, 1 mask, 4 attributes and 1 veg mask"""
    # Create a new channel to take into account the weather season, making it 17 channels
    if '2023_05'  in key:
        season_mask = np.zeros(array.shape[:-1]+ (1,))
    elif '2023_09' in key:
        season_mask = np.ones(array.shape[:-1]+ (1,))
    elif '2024_02' in key:
        season_mask = np.full(array.shape[:-1]+ (1,), 2)     

    array = np.concatenate((array, season_mask), axis=-1)

    # Delete the attribute channels (id, timestamp, ...)
    # Should have just 13 channels left
    array = np.delete(array, [11,12,13,14], axis=-1)

    arrays[key] = array

    filename = f"/home/k45848/multispectral-imagery-segmentation/data/clean_data/{key}.pkl"
    with open(filename, 'wb') as file:
        pickle.dump(array, file)


Check shape, of array, unique values e.t.c.

In [89]:
array.shape

(931, 932, 13)

In [90]:
for key, value in arrays.items():
    print(key, value.shape)

region_2_east_2023_05_31_final_data (746, 744, 13)
region_2_east_2024_02_25_final_data (746, 744, 13)
region_1_west_2023_09_26_final_data (931, 932, 13)
region_1_west_2024_02_28_final_data (931, 932, 13)
region_2_east_2023_09_28_final_data (746, 744, 13)
region_1_west_2023_05_31_final_data (931, 932, 13)


In [91]:
for key, value in arrays.items():
    print(np.unique(value[...,10], return_counts=True))

for key, value in arrays.items():
    print(np.unique(value[...,11], return_counts=True))
 
for key, value in arrays.items():
    print(key)
    print(np.unique(value[...,12], return_counts=True))

(array([0., 1., 2.]), array([211064, 342283,   1677]))
(array([0., 1., 2.]), array([245556, 302161,   7307]))
(array([0., 1., 2.]), array([423854, 436952,   6886]))
(array([0., 1., 2.]), array([421453, 439060,   7179]))
(array([0., 1., 2.]), array([215878, 332337,   6809]))
(array([0., 1., 2.]), array([421453, 442701,   3538]))
(array([0., 1., 2., 3., 5.]), array([211064,   1159,    316,    202, 342283]))
(array([0., 1., 2., 3., 5.]), array([245556,   6706,    399,    202, 302161]))
(array([0., 1., 2., 3., 4., 5.]), array([423854,   4149,    861,    445,   1431, 436952]))
(array([0., 1., 2., 3., 4., 5.]), array([421453,   4442,    861,    445,   1431, 439060]))
(array([0., 1., 2., 3., 5.]), array([215878,   6208,    399,    202, 332337]))
(array([0., 1., 2., 3., 4., 5.]), array([421453,   1011,    651,    445,   1431, 442701]))
region_2_east_2023_05_31_final_data
(array([0.]), array([555024]))
region_2_east_2024_02_25_final_data
(array([2.]), array([555024]))
region_1_west_2023_09_26_f

Split arrays into patches and distribute patches randomly to train and eval directories

In [101]:
def to_patches(file_path: str, patch_size: int, stride: int) -> None:
    """
    Convert an array to patches of a given size and stride and save them to train or eval directory.

    Parameters:
    - file_path (str): Path to the input array file.
    - patch_size (int): Size of each patch.
    - stride (int): Stride between patches.
    - train_dir (str): Directory to save train patches.
    - eval_dir (str): Directory to save eval patches.
    """
    array = load_array(file_path)
    file_name = os.path.splitext(os.path.basename(file_path))[0]  # Get file name without extension

    height, width, _ =  array.shape
    patch_height, patch_width = patch_size, patch_size
    stride_height, stride_width = stride, stride

    # Calculate the number of patches in each dimension
    num_patches_height = (height - patch_height) // stride_height + 1
    num_patches_width = (width - patch_width) // stride_width + 1

    # Determine the directory based on a random choice for train or eval
    base_dir =os.path.dirname(file_path)
    train_dir = f"{base_dir}/train/"
    eval_dir = f"{base_dir}/eval/"

    # dir = train_dir if random.choice([True, False]) else eval_dir
    # Determine the directory based on the 80:20 split
    dir = train_dir if random.random() < 0.8 else eval_dir
    # os.makedirs(dir)

    for i in range(num_patches_height):
        for j in range(num_patches_width):
            start_i = i * stride_height
            start_j = j * stride_width
            patch = array[start_i:start_i + patch_height, start_j:start_j + patch_width, :]

            patch_filename = f"{dir}{file_name}_{i* num_patches_width + j + 1}.pkl"
             
            with open(patch_filename, 'wb') as f:
                pickle.dump(patch, f)

def count_files_in_directory(directory: str) -> int:
    return len([name for name in os.listdir(directory)])



In [99]:
folder_path = "/home/k45848/multispectral-imagery-segmentation/data/clean_data"
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith(".pkl")]
file_paths

for path in file_paths:
    to_patches(path, 64, 16)

dirs = ["/home/k45848/multispectral-imagery-segmentation/data/clean_data/train",
       "/home/k45848/multispectral-imagery-segmentation/data/clean_data/eval"]

for dir in dirs:
    count_files_in_directory(dir)

### Issues
- Not sure the train/eval split went the way I wanted (randomised)
- Still can't count the files in each dir
Maybe just do it like before (first split to patches, then copy)

In [102]:
for dir in dirs:
    count_files_in_directory(dir)